# ANIME DATASET ANALYSIS USING SQL

In [2]:
import pandas as pd
import sqlite3

#reading the CSV
df = pd.read_csv('/Users/saan/Downloads/anime-list.csv')
#connecting to a database
conn = sqlite3.connect("Anime.db") 
#storing table in the database
df.to_sql('anime_list', conn)

## Exploring Data

In [96]:
sql_string = "SELECT * FROM anime_list"
df = pd.read_sql(sql_string, conn)

df.head()


,index,Unnamed: 0,name,studio,theme,tags,source,rating,year,demographic,status,eps,eps_avg_duration_in_min,rated_by
0,0,0,Shingeki no Kyojin,Wit Studio,"Gore, Military, Survival","Action, Drama",Manga,8.53,2013.0,Shounen,Finished,25.0,24.0,3.4M
1,1,1,Death Note,Madhouse,Psychological,"Supernatural, Suspense",Manga,8.63,2006.0,Shounen,Finished,37.0,23.0,3.4M
2,2,2,Fullmetal Alchemist: Brotherhood,Bones,Military,"Action, Adventure, Drama, Fantasy",Manga,9.14,2009.0,Shounen,Finished,64.0,24.0,2.9M
3,3,3,Boku no Hero Academia,Bones,"School, Super Power",Action,Manga,7.95,2016.0,Shounen,Finished,13.0,24.0,2.6M
4,4,4,Naruto,Studio Pierrot,Martial Arts,"Action, Adventure, Fantasy",Manga,7.97,2002.0,Shounen,Finished,220.0,23.0,2.5M


In [71]:
# Checking data types of columns in the table
sql_string = "pragma table_info('anime_list')"
df = pd.read_sql(sql_string, conn)

df


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Unnamed: 0,INTEGER,0,None,0
2,2,name,TEXT,0,None,0
3,3,studio,TEXT,0,None,0
4,4,theme,TEXT,0,None,0
5,5,tags,TEXT,0,None,0
6,6,source,TEXT,0,None,0
7,7,rating,REAL,0,None,0
8,8,year,REAL,0,None,0
9,9,demographic,TEXT,0,None,0


In [246]:
# Checking number of entries

sql_string = "SELECT COUNT(*) FROM anime_list"
df = pd.read_sql(sql_string, conn)

df.head()

,COUNT(*)
0,3005


In [245]:
# Count of animes that have not been rated

sql_string = sql_string = """\
SELECT COUNT(*) 
FROM anime_list 
WHERE rating IS NULL
"""
df = pd.read_sql(sql_string, conn)

df.head()

,COUNT(*)
0,1193


## Possible areas for analysis

1. Top 20 rated animes 
2. Popularity based on the original source the anime is based on (manga/light novel/original) 
3. Production studios with the most animes 
4. Relationship between release of anime and rating (finished or still airing?)
5. Most popular genre by demographic 

## TOP RATED ANIMES

In [244]:
sql_string = """\
SELECT *
FROM anime_list 
ORDER BY rating DESC
LIMIT 20
"""
df = pd.read_sql(sql_string, conn)

df.head()

,index,Unnamed: 0,name,studio,theme,tags,source,rating,year,demographic,status,eps,eps_avg_duration_in_min,rated_by
0,2,2,Fullmetal Alchemist: Brotherhood,Bones,Military,"Action, Adventure, Drama, Fantasy",Manga,9.14,2009.0,Shounen,Finished,64.0,24.0,2.9M
1,114,114,Gintama¬∞,Bandai Namco Pictures,"Gag Humor, Historical, Parody, Samurai","Action, Comedy, Sci-Fi",Manga,9.09,2015.0,Shounen,Finished,51.0,24.0,526K
2,119,119,Spy x Family,CloverWorks,Childcare,"Action, Comedy",Manga,9.09,2022.0,Shounen,Airing,12.0,24.0,511K
3,19,19,Shingeki no Kyojin Season 3 Part 2,Wit Studio,"Gore, Military, Survival","Action, Drama",Manga,9.08,2019.0,Shounen,Finished,10.0,23.0,1.8M
4,6,6,Hunter x Hunter (2011),Madhouse,Shounen,"Action, Adventure, Fantasy",Manga,9.05,2011.0,Shounen,Finished,148.0,23.0,2.3M


The table above shows the top rated animes on MyAnimeList.

Most of the animes in the table above share the "Action" tag with some other tags which shows the popularity of action in anime combined with another genre such as comedy, slice-of-life etc. Action seems to be especially popular in fun slice-of-life animes.

Spy x Family is rated incredibly high despite the fact that it was released in the year 2022 and it is still airing. The show combines Action with Comedy which might be why it is popular. This high rating is also likely since it is rated by only 511k people unlike the others which are rated by millions of users.

# Anime source

In [247]:
# Number of distinct sources for animes

sql_string = """\
SELECT DISTINCT count(source) AS source_count
FROM anime_list
"""
df = pd.read_sql(sql_string, conn)

df.head()

,source_count
0,2510


In [248]:
# Top 5 sources for animes

sql_string = """\
SELECT source, COUNT(source) AS anime_count 
FROM anime_list 
GROUP BY source 
ORDER BY anime_count DESC
"""

df = pd.read_sql(sql_string, conn)

df.head()

,source,anime_count
0,Manga,1181
1,Original,850
2,Game,102
3,Other,84
4,4-koma manga,66


A high number of animes are based off of mangas followed by many original creations. Let's look at the highest rated anime in the top 5 categories

In [112]:
# top rated anime in each of the top 5 sources

sql_string = """\
SELECT source, 
       COUNT(source) AS anime_count,
       rating,
       ROW_NUMBER() OVER(PARTITION BY rating) AS rank_in_source, 
       name,
       studio
FROM (SELECT source, name, rating, studio FROM anime_list ORDER BY rating DESC) 
GROUP BY source 
ORDER BY anime_count DESC
"""

df = pd.read_sql(sql_string, conn)

df.head()

,source,anime_count,rating,rank_in_source,name,studio
0,Manga,1181,9.14,1,Fullmetal Alchemist: Brotherhood,Bones
1,Original,850,8.50,1,Samurai Champloo,Manglobe
2,Game,102,7.84,1,Gungrave,Madhouse
3,Other,84,8.02,1,Pui Pui Molcar,Shin-Ei Animation
4,4-koma manga,66,8.06,1,"Komi-san wa, Comyushou desu.",OLM


The table above showcases that Manga is the most popular source for animes and the highest rated anime (Fullmetal Alchemist as seen before) is based off of a manga.

In [183]:
# top 20 animes in manga and original source and their ratings 

In [226]:
# Top 20 Manga based animes

sql_string = """\
SELECT source,
       name,
       tags,
       rating,
       rated_by
FROM anime_list
WHERE source IS "Manga"
ORDER BY rating DESC
LIMIT 20
"""
df = pd.read_sql(sql_string, conn)

df.head()

,source,name,tags,rating,rated_by
0,Manga,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy",9.14,2.9M
1,Manga,Gintama¬∞,"Action, Comedy, Sci-Fi",9.09,526K
2,Manga,Spy x Family,"Action, Comedy",9.09,511K
3,Manga,Shingeki no Kyojin Season 3 Part 2,"Action, Drama",9.08,1.8M
4,Manga,Hunter x Hunter (2011),"Action, Adventure, Fantasy",9.05,2.3M


In [225]:
# Top 20 Original animes

sql_string = """\
SELECT source,
       name,
       tags,
       rating,
       rated_by
FROM anime_list
WHERE source IS "Original"
ORDER BY rating DESC
LIMIT 20
"""
df = pd.read_sql(sql_string, conn)

df.head()

,source,name,tags,rating,rated_by
0,Original,Samurai Champloo,"Action, Adventure, Comedy",8.50,1.1M
1,Original,Mononoke,"Fantasy, Horror, Mystery, Supernatural",8.43,274K
2,Original,Shoujo Kakumei Utena,"Comedy, Drama, Fantasy, Mystery",8.18,187K
3,Original,Kaleido Star,"Comedy, Drama, Fantasy",7.92,78K
4,Original,Yuusha-Ou GaoGaiGar,"Adventure, Sci-Fi",7.83,19K


Top animes based on manga seem to be more highly rated than original animes.
Fullmetal Alchemist Brotherhood is performing really well with a lot of likes by viewers
Samurai Champloo seems to be performing really well based on it being an original anime - viewers love it.

# Studios and Animes

In [217]:
# Checking studios with the most animes (excluding those with unknown studios)

sql_string = """\
SELECT studio,
    COUNT(studio) AS animes_by_studio,
    AVG(rating)
FROM anime_list
WHERE studio IS NOT "Unknown"
GROUP BY studio
ORDER BY animes_by_studio DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,studio,animes_by_studio,AVG(rating)
0,Toei Animation,177,7.056815
1,Studio Deen,86,7.145395
2,Madhouse,82,7.450000
3,OLM,75,6.764444
4,J.C.Staff,69,7.336471


All of these studios besides Madhouse does not have a top rated anime

In [219]:
# Ratings of animes over the years of the most popular studio as of 2022 - Bones

sql_string = """\
SELECT studio,
       name,
       source,
       rating,
       year
FROM anime_list
WHERE studio IS "Bones"
ORDER BY year DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,studio,name,source,rating,year
0,Bones,Vanitas no Karte Part 2,Manga,8.20,2022.0
1,Bones,Boku no Hero Academia 5th Season,Manga,7.44,2021.0
2,Bones,Vanitas no Karte,Manga,7.91,2021.0
3,Bones,Boku no Hero Academia 4th Season,Manga,7.96,2019.0
4,Bones,Bungou Stray Dogs 3rd Season,Manga,8.21,2019.0


In [231]:
# Madhouse in top 50 animes

sql_string = """\
SELECT studio,
       name,
       source,
       rating,
       year
FROM (SELECT name, rating, studio, source, year
      FROM anime_list
      ORDER BY rating DESC
      LIMIT 50)
WHERE studio IS "Bones"
ORDER BY rating DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,studio,name,source,rating,year
0,Bones,Fullmetal Alchemist: Brotherhood,Manga,9.14,2009.0


Looks like Bones Studio only has one anime in the top 50s, but it is the #1 anime with the highest rating. Bones Studio is putting effort into making this anime even better!

In [230]:
# Ratings of animes over the years of the 2nd most popular studio as of 2022 - Madhouse

sql_string = """\
SELECT studio,
       name,
       source,
       rating,
       year
FROM anime_list
WHERE studio IS "Madhouse"
ORDER BY year DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()



,studio,name,source,rating,year
0,Madhouse,Hakozume: Kouban Joshi no Gyakushuu,Manga,7.18,2022.0
1,Madhouse,Kyuuketsuki Sugu Shinu,Manga,7.23,2021.0
2,Madhouse,No Guns Life 2nd Season,Manga,7.06,2020.0
3,Madhouse,Diamond no Ace: Act II,Manga,8.25,2019.0
4,Madhouse,No Guns Life,Manga,6.86,2019.0


In [250]:
# Madhouse in top 50 animes

sql_string = """\
SELECT studio,
       name,
       source,
       rating,
       year,
       rated_by
FROM (SELECT name, rating, studio, source, year, rated_by
      FROM anime_list
      ORDER BY rating DESC
      LIMIT 50)
WHERE studio IS "Madhouse"
ORDER BY rating DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,studio,name,source,rating,year,rated_by
0,Madhouse,Hunter x Hunter (2011),Manga,9.05,2011.0,2.3M
1,Madhouse,Monster,Manga,8.83,2004.0,835K
2,Madhouse,Hajime no Ippo,Manga,8.74,2000.0,474K
3,Madhouse,Hajime no Ippo: New Challenger,Manga,8.66,2009.0,239K
4,Madhouse,Death Note,Manga,8.63,2006.0,3.4M


Madhouse has 6 animes in the top 50s which means they are performing really well. Madhouse' asset is Hunter x Hunter which has been popular since over a decade now.

In [249]:
# Anime status

sql_string = """\
SELECT status, COUNT(*) AS anime_count
FROM anime_list
GROUP BY status
ORDER BY anime_count DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,status,anime_count
0,Finished,2850
1,Airing,66
2,Not,33
3,ished,32
4,None,24


In [251]:
# Top 5 animes that are still airing and their ratings

sql_string = """\
SELECT status, name, rating, rated_by 
FROM anime_list 
WHERE status = "Airing" 
ORDER BY rating DESC
LIMIT 5
"""

df = pd.read_sql(sql_string, conn)

df.head()

,status,name,rating,rated_by
0,Airing,Spy x Family,9.09,511K
1,Airing,Kaguya-sama wa Kokurasetai: Ultra Romantic,8.97,414K
2,Airing,Kingdom 4th Season,8.69,20K
3,Airing,One Piece,8.63,1.8M
4,Airing,"Komi-san wa, Comyushou desu. 2nd Season",8.32,221K


In [252]:
# Top 5 animes that are finished and their ratings

sql_string = """\
SELECT status, name, rating, rated_by 
FROM anime_list 
WHERE status = "Finished" 
ORDER BY rating DESC
LIMIT 5
"""

df = pd.read_sql(sql_string, conn)

df.head()

,status,name,rating,rated_by
0,Finished,Fullmetal Alchemist: Brotherhood,9.14,2.9M
1,Finished,Gintama¬∞,9.09,526K
2,Finished,Shingeki no Kyojin Season 3 Part 2,9.08,1.8M
3,Finished,Hunter x Hunter (2011),9.05,2.3M
4,Finished,Gintama',9.05,464K


# Demographics

In [253]:
sql_string = """\
SELECT demographic, COUNT(name) AS total_animes
FROM anime_list 
GROUP BY demographic
ORDER BY total_animes DESC
"""

df = pd.read_sql(sql_string, conn)

df.head()

,demographic,total_animes
0,Kids,1376
1,Shounen,887
2,Seinen,372
3,Shoujo,317
4,Josei,53


Kids and Shounen seem to be the most popular demographic

In [237]:
# Top 5 shows for Kids

sql_string = """\
SELECT demographic, name, rating, tags
FROM anime_list 
WHERE demographic IS "Kids"
ORDER BY rating DESC
"""

df = pd.read_sql(sql_string, conn)

df.head()

,demographic,name,rating,tags
0,Kids,Tanoshii Muumin Ikka,8.19,"Adventure, Comedy, Fantasy, Slice, of, Life"
1,Kids,Pui Pui Molcar,8.02,Comedy
2,Kids,Ie Naki Ko,7.83,"Adventure, Drama, Slice, of, Life"
3,Kids,Digimon Adventure,7.77,"Action, Adventure, Comedy, Fantasy"
4,Kids,Pokemon XY&Z,7.67,"Action, Adventure, Comedy, Fantasy"


In [254]:
# Kids demographic animes in top 50s

sql_string = """\
SELECT demographic, name, rating
FROM (SELECT demographic, name, rating
      FROM anime_list
      ORDER BY rating DESC
      LIMIT 50)
WHERE demographic IS "Kids"
ORDER BY rating DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,demographic,name,rating


Even though kids animes are rated well, they are not highly popular.

In [178]:
# Top 5 shows for Shounen

sql_string = """\
SELECT demographic, name, rating
FROM anime_list 
WHERE demographic IS "Shounen"
ORDER BY rating DESC
"""

df = pd.read_sql(sql_string, conn)

df.head

,demographic,name,rating
0,Shounen,Fullmetal Alchemist: Brotherhood,9.14
1,Shounen,Gintama¬∞,9.09
2,Shounen,Spy x Family,9.09
3,Shounen,Shingeki no Kyojin Season 3 Part 2,9.08
4,Shounen,Hunter x Hunter (2011),9.05


In [255]:
# Shounen demographic animes in top 50s

sql_string = """\
SELECT demographic, name, rating
FROM (SELECT demographic, name, rating
      FROM anime_list
      ORDER BY rating DESC
      LIMIT 50)
WHERE demographic IS "Shounen"
ORDER BY rating DESC
"""
df = pd.read_sql(sql_string, conn)

df.head()

,demographic,name,rating
0,Shounen,Fullmetal Alchemist: Brotherhood,9.14
1,Shounen,Gintama¬∞,9.09
2,Shounen,Spy x Family,9.09
3,Shounen,Shingeki no Kyojin Season 3 Part 2,9.08
4,Shounen,Hunter x Hunter (2011),9.05


Shounen animes take 29 out of the 50 spots in the top 50 animes!
These animes tend to be highly popular among ages of all kinds

Check out some of the visualizations from this project here:
[Tableau Visualizations](https://public.tableau.com/views/AnimeDashboard_16736507314250/Dashboard1?:language=en-GB&:display_count=n&:origin=viz_share_link)